# Streamageddon
Spotify's armageddon investments and data science meet
## Preprocessing

In [3]:
%pip install pandas==2.3.2

Note: you may need to restart the kernel to use updated packages.


In [39]:
import pandas as pd

df_csv = pd.read_csv("artists.csv")
df_json = pd.read_json("spotify-artists.json")
df_json_normalized = pd.json_normalize(df_json["artists"])

id = df_json_normalized["id"]
genres = df_json_normalized["genres"]
followers_total = df_json_normalized["followers.total"]
popularity = df_json_normalized["popularity"]
name = df_json_normalized["name"]
# First image of an artist
image = df_json_normalized["images"][0][0]["url"]
data = {"SpotifyId": id,
        "Genres": genres,
        "Followers": followers_total,
        "Popularity": popularity,
        "Image": image}

df_json_final = pd.DataFrame(data)
df = df_json_final.merge(df_csv, on='SpotifyId', how='inner')
print(df.head())

                SpotifyId                               Genres  Followers  \
0  6XYvaoDGE0VmRt83Jss9Sn  [neo-psychedelic, psychedelic rock]    1316858   
1  5DDuwU1X2Aqdp3CxfXstRe                        [lo-fi indie]      27499   
2  7AZwAitWq1KcFoIJhRWb6V                         [noise rock]     122693   
3  5JLqvjW3Nyom2OsRUyFsS9  [art pop, noise rock, experimental]     188888   
4  1kpDT6GWeimdyCmkwvTPvt          [electronic, house, techno]       1987   

   Popularity                                              Image  \
0          49  https://i.scdn.co/image/ab6761610000e5ebce4d5d...   
1          33  https://i.scdn.co/image/ab6761610000e5ebce4d5d...   
2          29  https://i.scdn.co/image/ab6761610000e5ebce4d5d...   
3          34  https://i.scdn.co/image/ab6761610000e5ebce4d5d...   
4           7  https://i.scdn.co/image/ab6761610000e5ebce4d5d...   

                               Name      Date  
0  King Gizzard & the Lizard Wizard  20250725  
1                       Hotline 